In [7]:
!git clone https://github.com/Tommyhuy1705/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication.git

Cloning into 'Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication'...
fatal: cannot write keep file 'C:/Users/duong/Downloads/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/notebooks/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/.git/objects/pack/pack-4fd7b959265ac910e8144e796cba75fed8d5928e.keep': Filename too long
fatal: fetch-pack: invalid index-pack output


In [1]:
%cd Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication

[WinError 2] The system cannot find the file specified: 'Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication'
c:\Users\duong\Downloads\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\notebooks


C:\Users\duong\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
# Cài đặt package
!pip install -e .

Obtaining file:///C:/Users/duong/Downloads/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/notebooks



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: file:///C:/Users/duong/Downloads/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/notebooks does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
# Import module
from utils.helpers import load_config, save_model, train_model, train_model_kfold
from utils.model_evaluation import evaluate_model, draw_plot_evaluate, draw_plot_find_acc
from models.Triplet_Siamese_Similarity_Network import tSSN
from losses.triplet_loss import TripletLoss
from dataloader.tSSN_trainloader import SignatureTrainDataset

print("Package installed and imported successfully!")

Package installed and imported successfully!


In [4]:
import os
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader,Subset
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from sklearn.metrics import accuracy_score

#seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Transform chung
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),  # Đảm bảo ảnh 1 kênh xám
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 1 kênh -> 3 kênh
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = SignatureTrainDataset(
    org_dir=r'/kaggle/input/cedardataset/signatures/full_org',
    forg_dir=r'/kaggle/input/cedardataset/signatures/full_forg',
    transform=transform
)

train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, len(dataset) - train_size]
)

train_loader = DataLoader(train_dataset, num_workers=4, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, num_workers=4, batch_size=32, shuffle=False)

In [ ]:
# Kiểm tra Triplet Dataset
print(f"Dataset - Total triplets: {len(dataset)}")

anchor, positive, negative = dataset[0]

print(f"Anchor shape: {anchor.shape}")
print(f"Positive shape: {positive.shape}")
print(f"Negative shape: {negative.shape}")

print(f"Training dataset: {len(train_dataset)}")

anchor_train, positive_train, negative_train = train_dataset[0]

print(f"Anchor shape: {anchor_train.shape}")
print(f"Positive shape: {positive_train.shape}")
print(f"Negative shape: {negative_train.shape}")

print(f"Testing dataset: {len(test_dataset)}")

anchor_test, positive_test, negative_test = test_dataset[0]

print(f"Anchor shape: {anchor_test.shape}")
print(f"Positive shape: {positive_test.shape}")
print(f"Negative shape: {negative_test.shape}")

print("LOAD DATASET SUCCESSFULLY")
print(f"Train-Test Split Ratio: {len(train_dataset)/len(dataset)*100:.1f}% - {len(test_dataset)/len(dataset)*100:.1f}%")

In [ ]:
config = load_config(r'/kaggle/working/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/configs/config_tSSN.yaml')
print(config)
print("LOAD CONFIG SUCCESSFULLY")

In [ ]:
modes = ['euclidean', 'cosine', 'manhattan', 'learnable']
margins = np.arange(0.2, 1.01, 0.2)

results_dict:dict = {}

for mode in modes:
    if mode == 'learnable':
        print(f"\nTraining mode: {mode} | margin: learnable")

        loss_fn = TripletLoss(
                margin=margin,
                mode=mode,
                input_dim=config['model']['feature_dim']
        )
        
        mean_acc, mean_loss = train_model_kfold(
            config=config,
            loss_fn=loss_fn,
            dataset=train_dataset,
            k_folds=10
        )

        results_dict[f"{mode}"] = {
            'mean_acc': mean_acc,
            'mean_loss': mean_loss
        }
    else:
        for margin in margins:
            print(f"\nTraining mode: {mode} | margin: {margin:.1f}")
                    
            loss_fn = TripletLoss(
                margin=margin,
                mode=mode,
                input_dim=config['model']['feature_dim']
            )

            mean_acc, mean_loss = train_model_kfold(
                config=config,
                loss_fn=loss_fn,
                dataset=train_dataset,
                k_folds=10
            )
            results_dict[f"{mode}_{margin:.1f}"] = {
                'mean_acc': mean_acc,
                'mean_loss': mean_loss
            }   

In [ ]:
# Ở đây lấy results_dict vẽ plot với y là mean_acc, x là key
# xem model là tốt nhất
# từ đó lấy ra best paremter 
# Viết hàm vẽ plot đó vào trong file utils/model_evaluation.py

best_params = draw_plot_find_acc(results_dict)

In [ ]:
# Block này sẽ train ra model cuối cùng với các tham số tốt nhất

best_mode = best_params['mode']
best_margin = best_params['margin']
save_path = f"/kaggle/working/final"

model = tSSN(config['model']['backbone'], config['model']['feature_dim'])
device = torch.device(config['device'] if torch.cuda.is_available() else "cpu")
model.to(device)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0, 1])

loss_fn = TripletLoss(
    margin=0 if best_mode == 'learnable' else best_margin,
    mode=best_mode,
    input_dim=config['model']['feature_dim']
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config['training']['learning_rate'])

model, final_train_loss = train_model(
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    num_epochs=config['training']['num_epochs'],
    device=device,
    early_stop=config['training']['early_stop']
)

# Lưu model
save_model(model=model,
        dir= save_path,     
        optimizer=optimizer,
        avg_loss=final_train_loss,
        model_name=f"tSNN_{best_mode}_{best_margin:.1f}")
           

In [ ]:
# Đây sẽ là các block đánh giá các thông số của model cuối cùng, kèm theo đó là vẽ plot, cũng như test model
# Hàm đánh giá tui để trong utils/model_evaluation.py tên hàm vẫn như cũ ae chỉ cần gọi là đc
result = evaluate_model(model, best_mode, test_loader, device)

draw_plot_evaluate(result, "all")

In [ ]:
import kagglehub

kagglehub.login()

kagglehub.model_upload(
    handle="giahuytranviet/tSSN-verification-model/pyTorch/default",
    local_model_dir="/kaggle/working/final",
    version_notes="Upload latest model with best parameters",
)